In [ ]:
import json
import logging
from dataclasses import dataclass, field
from typing import Dict, Any, List
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

# Groq model initialization
GROQ_API_KEY = "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"
model = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama-3.2-3b-preview",
)

# Logging configuration
logging.basicConfig(
    level=logging.INFO,
    filename='execution.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Mock ServiceNow Ticket API Response
def get_servicenow_ticket(ticket_number: str) -> Dict[str, Any]:
    """Mock function to simulate fetching a ticket from ServiceNow API.
    
    Returns a realistic ServiceNow ticket structure based on ticket number.
    """
    ticket_templates = {
        "AD001": {
            "number": "AD001",
            "short_description": "Create new Active Directory group for Marketing team",
            "description": "Please create a new security group in Active Directory for the Marketing department. Group name should be 'MARKETING_TEAM'. Owner should be John Smith (jsmith@company.com).",
            "category": "Active Directory",
            "subcategory": "Group Management",
            "priority": "3 - Moderate",
            "assignment_group": "Identity Management",
            "requested_by": "Sarah Johnson (sjohnson@company.com)"
        },
        "M365002": {
            "number": "M365002",
            "short_description": "Verify Microsoft 365 license for new employee",
            "description": "New employee Michael Brown (mbrown@company.com) needs verification of Microsoft 365 E3 license assignment before onboarding.",
            "category": "Microsoft 365",
            "subcategory": "License Management",
            "priority": "2 - High",
            "assignment_group": "Cloud Services",
            "requested_by": "HR Onboarding Team (hronboarding@company.com)"
        },
        "AD003": {
            "number": "AD003",
            "short_description": "Check owner of Sales team AD group",
            "description": "Need to verify current owner of 'SALES_TEAM' Active Directory group and update if necessary.",
            "category": "Active Directory",
            "subcategory": "Group Management",
            "priority": "4 - Low",
            "assignment_group": "Identity Management",
            "requested_by": "Robert Chen (rchen@company.com)"
        }
    }
    
    # Return the matching ticket or a default one
    return ticket_templates.get(ticket_number, {
        "number": ticket_number,
        "short_description": "General IT request",
        "description": "No specific details provided",
        "category": "General",
        "subcategory": "Other",
        "priority": "4 - Low",
        "assignment_group": "IT Helpdesk",
        "requested_by": "Unknown"
    })

# Tool functions with docstrings
def parse_variables(query: str) -> Dict[str, str]:
    """Parse variables from the given query string.
    
    Args:
        query (str): The query string to parse.
    
    Returns:
        dict: Extracted variables with their values.
    """
    logging.debug(f"Parsing variables from query: {query}")
    
    # Simple parsing logic - in a real implementation, you'd use more sophisticated NLP
    variables = {}
    if "group name" in query.lower():
        variables["group_name"] = extract_between_quotes(query) or "DEFAULT_GROUP"
    if "owner" in query.lower():
        variables["owner"] = extract_email(query) or "admin@company.com"
    if "user" in query.lower():
        variables["user_id"] = extract_email(query) or "user@company.com"
    
    logging.info(f"Parsed variables: {variables}")
    return variables

def extract_between_quotes(text: str) -> str:
    """Helper to extract text between single or double quotes."""
    import re
    match = re.search(r'[\'"](.+?)[\'"]', text)
    return match.group(1) if match else None

def extract_email(text: str) -> str:
    """Helper to extract email addresses."""
    import re
    match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    return match.group(0) if match else None

def check_ad_group_existence(group_name: str) -> str:
    """Check if an Active Directory group exists.
    
    Args:
        group_name (str): The name of the AD group to check.
    
    Returns:
        str: A message indicating whether the AD group exists.
    """
    logging.debug(f"Checking existence of AD group: {group_name}")
    # Mock response - in real implementation, this would check AD
    return f"AD group '{group_name}' exists." if group_name.startswith(('SALES', 'IT')) else f"AD group '{group_name}' does not exist."

def check_owner_existence(owner_email: str) -> str:
    """Check if an owner exists.
    
    Args:
        owner_email (str): The email of the owner to check.
    
    Returns:
        str: A message indicating whether the owner exists.
    """
    logging.debug(f"Checking existence of owner: {owner_email}")
    # Mock response - in real implementation, this would check AD
    return f"Owner '{owner_email}' exists." if owner_email.endswith('@company.com') else f"Owner '{owner_email}' does not exist."

def create_ad_group(group_name: str, owner_email: str) -> str:
    """Create a new Active Directory group.
    
    Args:
        group_name (str): The name of the AD group to create.
        owner_email (str): The email of the owner for the new group.
    
    Returns:
        str: A message indicating the AD group was created.
    """
    logging.debug(f"Creating AD group: {group_name} with owner {owner_email}")
    # Mock response - in real implementation, this would create the group in AD
    return f"AD group '{group_name}' created successfully with owner '{owner_email}'."

def check_m365_license(user_email: str) -> str:
    """Check the Microsoft 365 license status for a user.
    
    Args:
        user_email (str): The email of the user to check.
    
    Returns:
        str: A message indicating the M365 license status.
    """
    logging.debug(f"Checking M365 license for user: {user_email}")
    # Mock response - in real implementation, this would check M365
    return f"User '{user_email}' has Microsoft 365 E3 license assigned." if user_email else "No license information found."

# Create agents
ad_agent = create_react_agent(
    model=model,
    tools=[parse_variables, check_ad_group_existence, check_owner_existence, create_ad_group],
    name="ad_expert",
    prompt="You are an Active Directory (AD) expert. Use the appropriate tool to process the query."
)

m365_agent = create_react_agent(
    model=model,
    tools=[check_m365_license],
    name="m365_expert",
    prompt="You are a Microsoft 365 (M365) expert. Use the appropriate tool to process the query."
)

# Create supervisor
workflow = create_supervisor(
    [ad_agent, m365_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing an AD expert and an M365 expert. "
        "Analyze ServiceNow tickets and route them appropriately. "
        "For tickets related to Active Directory (group creation, modification, etc.), use ad_agent. "
        "For tickets related to Microsoft 365 (license checks, etc.), use m365_agent. "
        "First parse the ticket details to extract required variables."
    )
)

# Define the state type for the workflow
@dataclass
class GraphState:
    ticket_number: str = ""
    ticket_data: Dict[str, Any] = field(default_factory=dict)
    next_step: str = ""
    process_complete: bool = False
    extracted_vars: Dict[str, Any] = field(default_factory=dict)
    tool_logs: List[Dict[str, Any]] = field(default_factory=list)

# Supervisor Node
def supervisor_node(state: GraphState) -> Dict[str, Any]:
    logging.info("Supervisor: Evaluating next step.")
    logging.debug(f"Current state: {state}")
    
    if state.process_complete:
        logging.info("Supervisor: Process complete. Ending.")
        return {"next_step": "end"}
    
    # If we don't have ticket data yet, fetch it
    if not state.ticket_data:
        logging.info(f"Supervisor: Fetching ticket {state.ticket_number} from ServiceNow")
        state.ticket_data = get_servicenow_ticket(state.ticket_number)
        logging.debug(f"Retrieved ticket data: {state.ticket_data}")
    
    # Determine which agent to use based on ticket category
    category = state.ticket_data.get("category", "").lower()
    
    if "active directory" in category:
        next_step = "ad_agent"
    elif "microsoft 365" in category:
        next_step = "m365_agent"
    else:
        logging.info("Supervisor: No relevant category found. Ending.")
        return {
            "next_step": "end",
            "process_complete": True,
            "ticket_data": state.ticket_data
        }
    
    logging.info(f"Supervisor: Next step is {next_step}.")
    return {
        "next_step": next_step,
        "ticket_data": state.ticket_data,
        "process_complete": state.process_complete
    }

# AD Agent Node
def ad_agent_node(state: GraphState) -> Dict[str, Any]:
    logging.info("AD Agent: Processing Active Directory request.")
    logging.debug(f"Current state: {state}")
    
    # Copy state fields to update them
    ticket_data = state.ticket_data.copy()
    tool_logs = state.tool_logs.copy()
    extracted_vars = state.extracted_vars.copy()
    
    # First parse variables if not already done
    if not extracted_vars:
        logging.info("AD Agent: Parsing variables from ticket description")
        description = ticket_data.get("description", "")
        extracted_vars = parse_variables(description)
        tool_logs.append({
            "tool_name": "parse_variables",
            "result": f"Extracted variables: {extracted_vars}"
        })
    
    # Determine which tools to run based on ticket description
    description = ticket_data.get("description", "").lower()
    
    if "create" in description and "group" in description:
        # Full group creation workflow
        group_name = extracted_vars.get("group_name", "DEFAULT_GROUP")
        owner_email = extracted_vars.get("owner", "admin@company.com")
        
        # Check if group exists
        existence_check = check_ad_group_existence(group_name)
        tool_logs.append({
            "tool_name": "check_ad_group_existence",
            "result": existence_check
        })
        
        # If group doesn't exist, create it
        if "does not exist" in existence_check:
            creation_result = create_ad_group(group_name, owner_email)
            tool_logs.append({
                "tool_name": "create_ad_group",
                "result": creation_result
            })
        else:
            tool_logs.append({
                "tool_name": "create_ad_group",
                "result": "Group already exists - no creation needed"
            })
        
        # Check owner in either case
        owner_check = check_owner_existence(owner_email)
        tool_logs.append({
            "tool_name": "check_owner_existence",
            "result": owner_check
        })
    
    elif "check owner" in description or "verify owner" in description:
        # Owner verification workflow
        owner_email = extracted_vars.get("owner", "admin@company.com")
        owner_check = check_owner_existence(owner_email)
        tool_logs.append({
            "tool_name": "check_owner_existence",
            "result": owner_check
        })
    
    else:
        # Default AD check
        group_name = extracted_vars.get("group_name", "DEFAULT_GROUP")
        existence_check = check_ad_group_existence(group_name)
        tool_logs.append({
            "tool_name": "check_ad_group_existence",
            "result": existence_check
        })
    
    logging.debug(f"Updated state: extracted_vars={extracted_vars}, tool_logs={tool_logs}")
    return {
        "next_step": "supervisor",
        "process_complete": True,
        "ticket_data": ticket_data,
        "extracted_vars": extracted_vars,
        "tool_logs": tool_logs
    }

# M365 Agent Node
def m365_agent_node(state: GraphState) -> Dict[str, Any]:
    logging.info("M365 Agent: Processing Microsoft 365 request.")
    logging.debug(f"Current state: {state}")
    
    # Copy state fields to update them
    ticket_data = state.ticket_data.copy()
    tool_logs = state.tool_logs.copy()
    extracted_vars = state.extracted_vars.copy()
    
    # Parse variables if not already done
    if not extracted_vars:
        logging.info("M365 Agent: Parsing variables from ticket description")
        description = ticket_data.get("description", "")
        extracted_vars = parse_variables(description)
        tool_logs.append({
            "tool_name": "parse_variables",
            "result": f"Extracted variables: {extracted_vars}"
        })
    
    # Perform license check
    user_email = extracted_vars.get("user_id", "user@company.com")
    license_check = check_m365_license(user_email)
    tool_logs.append({
        "tool_name": "check_m365_license",
        "result": license_check
    })
    
    logging.debug(f"Updated state: extracted_vars={extracted_vars}, tool_logs={tool_logs}")
    return {
        "next_step": "supervisor",
        "process_complete": True,
        "ticket_data": ticket_data,
        "extracted_vars": extracted_vars,
        "tool_logs": tool_logs
    }

# Build graph
builder = StateGraph(GraphState)
builder.add_node("supervisor", supervisor_node)
builder.add_node("ad_agent", ad_agent_node)
builder.add_node("m365_agent", m365_agent_node)
builder.add_edge(START, "supervisor")
builder.add_conditional_edges(
    "supervisor",
    lambda state: state.next_step,
    {
        "ad_agent": "ad_agent",
        "m365_agent": "m365_agent",
        "end": END
    }
)
builder.add_edge("ad_agent", "supervisor")
builder.add_edge("m365_agent", "supervisor")
graph = builder.compile()

# Format tool logs for final output
def format_tool_logs(tool_logs: List[Dict[str, Any]]) -> str:
    if not tool_logs:
        return "No tools were executed."
    output = ["Execution Log:"]
    for i, log in enumerate(tool_logs, start=1):
        output.append(f"{i}. {log['tool_name']}: {log['result']}")
    return "\n".join(output)

# Format final response with ticket info
def format_final_response(ticket_data: Dict[str, Any], tool_logs: List[Dict[str, Any]]) -> str:
    response = [
        f"Ticket: {ticket_data.get('number', 'UNKNOWN')}",
        f"Category: {ticket_data.get('category', 'Unknown')}",
        f"Description: {ticket_data.get('short_description', 'No description')}",
        "",
        format_tool_logs(tool_logs),
        "",
        "=== Resolution ===",
        "All requested actions completed successfully."
    ]
    return "\n".join(response)

# Run test
if __name__ == "__main__":
    # Configure logging to also output to console for visibility
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    logging.getLogger().addHandler(console_handler)

    # Test case 1: Active Directory group creation
    print("\n=== Testing AD Group Creation Ticket ===")
    initial_input_ad = {
        "ticket_number": "AD001",
        "next_step": "supervisor"
    }
    
    logging.info("Starting execution for AD group creation ticket...")
    result_ad = graph.invoke(initial_input_ad)
    print(format_final_response(result_ad["ticket_data"], result_ad["tool_logs"]))

    # Test case 2: Microsoft 365 license check
    print("\n=== Testing M365 License Check Ticket ===")
    initial_input_m365 = {
        "ticket_number": "M365002",
        "next_step": "supervisor"
    }
    
    logging.info("Starting execution for M365 license check ticket...")
    result_m365 = graph.invoke(initial_input_m365)
    print(format_final_response(result_m365["ticket_data"], result_m365["tool_logs"]))

    # Test case 3: AD Owner verification
    print("\n=== Testing AD Owner Verification Ticket ===")
    initial_input_ad2 = {
        "ticket_number": "AD003",
        "next_step": "supervisor"
    }
    
    logging.info("Starting execution for AD owner verification ticket...")
    result_ad2 = graph.invoke(initial_input_ad2)
    print(format_final_response(result_ad2["ticket_data"], result_ad2["tool_logs"]))

In [ ]:
import os
import json
import logging
import yaml
from typing import Dict, Any, List, Optional
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END

# Load environment variables
load_dotenv()

# Configuration and Logging - Fixed duplicate logging
logger = logging.getLogger(__name__)
logger.handlers.clear()  # Clear existing handlers
logger.propagate = False  # Prevent propagation to root logger

if not logging.getLogger().hasHandlers():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[logging.FileHandler('execution.log'), logging.StreamHandler()]
    )

# Configure logging for HTTP clients
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("http.client").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

class Config:
    def __init__(self):
        self.agent_config = self._load_agent_config()
        self.use_llm_routing = False
        self.groq_api_key = os.getenv('GROQ_API_KEY')
        
        if self.groq_api_key:
            try:
                from langchain_groq import ChatGroq
                self.llm = ChatGroq(
                    api_key=self.groq_api_key,
                    model_name="llama3-70b-8192",
                    temperature=0
                )
                self.use_llm_routing = True
                logger.info("Using LLM-powered routing")
            except ImportError:
                logger.warning("Groq package not available")

    def _load_agent_config(self) -> Dict[str, Any]:
        try:
            with open('agent_config.yaml') as f:
                return yaml.safe_load(f)
        except FileNotFoundError:
            return {
                'agents': {
                    'ad': {'tools': ["check_ad_group", "create_ad_group"]},
                    'm365': {'tools': ["check_m365_license"]}
                }
            }

config = Config()

# Tool Definitions
@tool
def check_ad_group(group_name: str) -> Dict[str, Any]:
    """Check if an Active Directory group exists."""
    logger.info(f"Checking AD group: {group_name}")
    return {
        "status": "success",
        "exists": group_name.startswith(('SALES', 'IT')),
        "group_name": group_name
    }

@tool
def create_ad_group(group_name: str, owner_email: str) -> Dict[str, Any]:
    """Create a new Active Directory group."""
    logger.info(f"Creating AD group: {group_name}")
    return {
        "status": "success",
        "created": True,
        "group_name": group_name,
        "owner": owner_email
    }

@tool
def check_m365_license(user_email: str) -> Dict[str, Any]:
    """Check Microsoft 365 license status."""
    logger.info(f"Checking M365 license for: {user_email}")
    return {
        "status": "success",
        "licensed": True,
        "user_email": user_email,
        "license_type": "E3"
    }

# State Schema using Pydantic
class GraphState(BaseModel):
    ticket_number: str = ""
    short_description: str = ""
    next_step: str = "supervisor"
    process_complete: bool = False
    tool_logs: List[Dict[str, Any]] = []

    def to_dict(self) -> Dict[str, Any]:
        return self.dict()
    
    @classmethod
    def from_dict(cls, data: Optional[Dict[str, Any]]) -> 'GraphState':
        if data is None:
            return cls()
        return cls.parse_obj(data)

# Routing Logic
def get_supervisor_node():
    if config.use_llm_routing:
        def llm_supervisor_node(state: Dict[str, Any]) -> Dict[str, Any]:
            try:
                state_obj = GraphState.from_dict(state)
                if state_obj.process_complete:
                    state_obj.next_step = "end"
                    return state_obj.to_dict()

                prompt = ChatPromptTemplate.from_template("""
                Analyze this IT ticket and route it:
                Ticket: {ticket_number}
                Description: {description}
                Options: ad_agent (AD issues) | m365_agent (M365 issues) | end
                Respond ONLY with 'ad_agent', 'm365_agent', or 'end'.
                """)

                routing_chain = prompt | config.llm | StrOutputParser()
                decision = routing_chain.invoke({
                    "ticket_number": state_obj.ticket_number,
                    "description": state_obj.short_description
                }).strip().lower()

                valid_options = {"ad_agent", "m365_agent", "end"}
                state_obj.next_step = decision if decision in valid_options else "end"
                return state_obj.to_dict()
            except Exception as e:
                logger.error(f"Error in llm_supervisor_node: {str(e)}", exc_info=True)
                return GraphState().to_dict()  # Return default state on error

        return llm_supervisor_node
    else:
        logger.info("Using simple keyword-based routing")
        def simple_supervisor_node(state: Dict[str, Any]) -> Dict[str, Any]:
            try:
                state_obj = GraphState.from_dict(state)
                if state_obj.process_complete:
                    state_obj.next_step = "end"
                    return state_obj.to_dict()

                desc = state_obj.short_description.lower()
                state_obj.next_step = (
                    "ad_agent" if "active directory" in desc
                    else "m365_agent" if "microsoft 365" in desc
                    else "end"
                )
                return state_obj.to_dict()
            except Exception as e:
                logger.error(f"Error in simple_supervisor_node: {str(e)}", exc_info=True)
                return GraphState().to_dict()  # Return default state on error

        return simple_supervisor_node

# Agent Nodes
def ad_agent_node(state: Dict[str, Any]) -> Dict[str, Any]:
    try:
        state_obj = GraphState.from_dict(state)
        logger.info(f"AD Agent processing: {state_obj.ticket_number}")
        desc = state_obj.short_description.lower()
        group_name = "Sales_Team" if "sales" in desc else "Default_Group"

        state_obj.tool_logs.append({
            "tool": "check_ad_group",
            "result": check_ad_group.invoke({"group_name": group_name})
        })

        if "create" in desc:
            state_obj.tool_logs.append({
                "tool": "create_ad_group",
                "result": create_ad_group.invoke({
                    "group_name": group_name,
                    "owner_email": "admin@company.com"
                })
            })

        state_obj.next_step = "end"
        state_obj.process_complete = True
        return state_obj.to_dict()
    except Exception as e:
        logger.error(f"Error in ad_agent_node: {str(e)}", exc_info=True)
        return GraphState().to_dict()

def m365_agent_node(state: Dict[str, Any]) -> Dict[str, Any]:
    try:
        state_obj = GraphState.from_dict(state)
        logger.info(f"M365 Agent processing: {state_obj.ticket_number}")
        state_obj.tool_logs.append({
            "tool": "check_m365_license",
            "result": check_m365_license.invoke({"user_email": "user@company.com"})
        })

        state_obj.next_step = "end"
        state_obj.process_complete = True
        return state_obj.to_dict()
    except Exception as e:
        logger.error(f"Error in m365_agent_node: {str(e)}", exc_info=True)
        return GraphState().to_dict()

# Workflow Construction
def build_workflow():
    builder = StateGraph(GraphState)

    builder.add_node("supervisor", get_supervisor_node())
    builder.add_node("ad_agent", ad_agent_node)
    builder.add_node("m365_agent", m365_agent_node)

    builder.add_edge(START, "supervisor")
    builder.add_conditional_edges(
        "supervisor",
        lambda state: GraphState.from_dict(state).next_step,
        {
            "ad_agent": "ad_agent",
            "m365_agent": "m365_agent",
            "end": END
        }
    )
    builder.add_edge("ad_agent", END)
    builder.add_edge("m365_agent", END)

    return builder.compile()

# Helper Functions
def format_output(state: Optional[Dict[str, Any]]) -> str:
    try:
        state_obj = GraphState.from_dict(state)
        output = [
            f"\nTicket: {state_obj.ticket_number}",
            f"Description: {state_obj.short_description}",
            "\nTool Execution Results:",
            "-" * 40
        ]
        for log in state_obj.tool_logs:
            output.append(
                f"Tool: {log['tool']}\n"
                f"Result: {json.dumps(log['result'], indent=2)}"
            )
        output.append(f"\nStatus: {'COMPLETED' if state_obj.process_complete else 'PENDING'}")
        return "\n".join(output)
    except Exception as e:
        logger.error(f"Error formatting output: {str(e)}", exc_info=True)
        return f"Error formatting output: {str(e)}"

# Main Execution
if __name__ == "__main__":
    try:
        workflow = build_workflow()

        test_cases = [
            ("AD001", "Create new Active Directory group for Sales team"),
            ("M365001", "Verify Microsoft 365 license for user@company.com"),
            ("UNK001", "Printer setup request"),
            ("AD002", "Check permissions for IT_ADMINS group")
        ]

        for ticket_num, desc in test_cases:
            logger.info(f"\nProcessing Ticket {ticket_num}")
            try:
                initial_state = GraphState(
                    ticket_number=ticket_num,
                    short_description=desc,
                    next_step="supervisor",
                    process_complete=False,
                    tool_logs=[]
                )
                result = workflow.invoke(initial_state.to_dict())
                print(format_output(result))
                print("\n" + "=" * 60)
            except Exception as e:
                logger.error(f"Error processing {ticket_num}: {str(e)}", exc_info=True)
                print(f"Failed to process ticket {ticket_num}: {str(e)}")
                print("\n" + "=" * 60)
    except Exception as e:
        logger.error(f"Failed to build workflow: {str(e)}", exc_info=True)

2025-03-27 01:13:59,784 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-27 01:13:59,784 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Error formatting output: 'NoneType' object has no attribute 'get'
Traceback (most recent call last):
  File "C:\Users\mahid\AppData\Local\Temp\ipykernel_16388\1680158210.py", line 283, in format_output
    state_obj = GraphState.from_dict(state)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mahid\AppData\Local\Temp\ipykernel_16388\1680158210.py", line 151, in from_dict
    ticket_number=data.get("ticket_number", ""),
                  ^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'get'


Error formatting output: 'NoneType' object has no attribute 'get'



2025-03-27 01:14:00,079 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-27 01:14:00,079 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Error formatting output: 'NoneType' object has no attribute 'get'
Traceback (most recent call last):
  File "C:\Users\mahid\AppData\Local\Temp\ipykernel_16388\1680158210.py", line 283, in format_output
    state_obj = GraphState.from_dict(state)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mahid\AppData\Local\Temp\ipykernel_16388\1680158210.py", line 151, in from_dict
    ticket_number=data.get("ticket_number", ""),
                  ^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'get'


Error formatting output: 'NoneType' object has no attribute 'get'



2025-03-27 01:14:00,368 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-27 01:14:00,368 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Error formatting output: 'NoneType' object has no attribute 'get'
Traceback (most recent call last):
  File "C:\Users\mahid\AppData\Local\Temp\ipykernel_16388\1680158210.py", line 283, in format_output
    state_obj = GraphState.from_dict(state)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mahid\AppData\Local\Temp\ipykernel_16388\1680158210.py", line 151, in from_dict
    ticket_number=data.get("ticket_number", ""),
                  ^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'get'


Error formatting output: 'NoneType' object has no attribute 'get'



2025-03-27 01:14:00,668 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-27 01:14:00,668 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Error formatting output: 'NoneType' object has no attribute 'get'
Traceback (most recent call last):
  File "C:\Users\mahid\AppData\Local\Temp\ipykernel_16388\1680158210.py", line 283, in format_output
    state_obj = GraphState.from_dict(state)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mahid\AppData\Local\Temp\ipykernel_16388\1680158210.py", line 151, in from_dict
    ticket_number=data.get("ticket_number", ""),
                  ^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'get'


Error formatting output: 'NoneType' object has no attribute 'get'

